<a href="https://colab.research.google.com/github/sadnow/RIFE-Google-Colab-Simplified/blob/main/AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit - Upscaling & Interpolation using Real-ESRGAN+RIFE

Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

# Mount Google Drive & Install Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#install dependencies

############# Clone Real-ESRGAN and enter the Real-ESRGAN
print("Installing libraries for Real-ESRGAN upscaling.")
%cd /content/
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop
# Download the pre-trained model
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
%cd /content/
print("Finished Installing libraries for Real-ESRGAN upscaling.")
#############
#############
%cd /content/
print("Installing libraries for motion smoothing.")
!git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
!gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
!7z e RIFE_trained_model_HDv2.zip
!mkdir /content/RIFE/train_log
!mv *.pkl /content/RIFE/train_log/
%cd /content/RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt
%cd /content/
print("Done.")
%cd /content/RIFE/
print("Finsihed Installing libraries for motion smoothing.")
%cd /content/
#############
!pip install ffmpeg-python


# Real-ESRGAN Video Upscaling


In [ ]:
%cd /content/
#frame_rate = "23.976" #@param {type:"string"}
#start_time = "00:00:00.000"
#end_time = "00:00:05.000" #@param {type:"string"}
video_file_path='/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh.mp4' #@param {type:"string"}
target_path='/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh-upscaled.mp4' #@param {type:"string"}
target_fps='24'#@param {type:"string"}
scale_value="3.5" #@param {type:"string"}
#@markdown Default `scale_value` is either 3.5 or 4

#@markdown Run RIFE below to fill in empty frames.
#@markdown Upscaled video saved as `/content/output.mp4`

######Extract Individual frames
#* [*Example*](https://yuju.pw/y/36pP.png) *of output folder*
import os, sys, re
output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
#os.environ['startTime'] = start_time
#os.environ['endTime'] = end_time
os.environ['frameRate'] = target_fps
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

###################split frames
!mkdir "/content/Real-ESRGAN/upload"
!ffmpeg -hide_banner -i "$inputFile" -r "$frameRate"/1 "/content/Real-ESRGAN/upload"/frame%04d.png
#!ffmpeg -hide_banner -i "$inputFile" -r "$frameRate"/1 "/content/Real-ESRGAN/upload"/frame%04d.png
#!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -r "$frameRate"/1 "$outputPath"/"Extracted Frames"/frame%04d.png
####################


#upload images
import os
from google.colab import files
import shutil

upload_folder = 'upload'
result_folder = 'results'

if os.path.isdir(upload_folder):
    shutil.rmtree(upload_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)
os.mkdir(upload_folder)
os.mkdir(result_folder)




###########run upscaler
%cd /content/Real-ESRGAN
#!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale $scale_value --outscale $scale_value --half --face_enhance
!python inference_realesrgan.py --model_path experiments/pretrained_models/RealESRGAN_x4plus.pth --input upload --netscale $scale_value --outscale $scale_value --half --face_enhance

##########

######re-encode video from frames
%cd /content/Real-ESRGAN/results
#!ffmpeg -r 1/5 -pattern_type glob -i '*.png' -c:v libx264 /content/output.mp4   # x264 video
!ffmpeg -r $target_fps -pattern_type glob -i '*.png' -c:v libx264 $target_path   # x264 video



# Motion interpolation (RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."


In [ ]:
%cd /content/RIFE/
# @title Settings
input_path='/content/output.mp4' #@param {type:"string"}
#target_fps=360#@param {type:"raw"}
length_multiplier=2#@param {type:"integer"}
#@markdown
#@markdown `length_multiplier` might double, triple, or quadruple your video. Or square it? It's mysterious.
#@markdown
#@markdown ---
#@markdown Note: Your output will look like `original_2x_60fps.mp4` and can be found in your `input_path`.
#@markdown
#@markdown Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)

!python3 /content/RIFE/inference_video.py --fps=$target_fps --exp=$length_multiplier --video=$input_path --skip
%cd /content/

# x265 Conversion/Compression (Experimental, Very Fast)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss.

**Note: You manually need to input your desired compress_path (sorry) - will update this in the future w/ a checkbox to automatically determine your outputted interpolated file.**

In [ ]:
compress_path='/content/drive/MyDrive/VQLIPSE/videos/output_2X_60fps.mp4' #@param {type:"string"}
outputStr = '_x265-converted.mp4'
constant_quality=26#@param {type:"slider", min:20, max:50, step:1}


#!ffmpeg -i $compress_path -vcodec hevc_nvenc -rc:v vbr_hq -cq $constant_quality $compress_path$outputStr -async 1 -vsync 1
!ffmpeg -i $compress_path -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $compress_path$outputStr 

#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality

#@markdown Note: Your output will look like `original_2x_60fps.mp4__x265-converted.mp4` and can be found in your `compress_path`.